# Environment Test
Run this notebook as an initial test that the docker environment is working as expected.  
The goal is to confirm that earthscopestraintools is installed properly, as well as the fortran program 'hartid' which is used to generate tidal corrections.

If all the cells run, and the last cell makes a plot with sinusoids, then this environment should be working.

In [ ]:
#confirm earthscopestraintools is installed and which version
!pip show earthscopestraintools

In [ ]:
#confirm the spotl program hartid is installed and available in your path
! which hartid

In [ ]:
#confirm baytap08 is installed and available in your path
! which baytap08

In [ ]:
# import some basic modules from earthscopestraintools and configure logging to appear in the notebook
from earthscopestraintools.mseed_tools import ts_from_mseed
from earthscopestraintools.gtsm_metadata import GtsmMetadata
import logging
logger = logging.getLogger()
logging.basicConfig(
        format="%(message)s", level=logging.INFO
    )

In [ ]:
#download some metadata and 10 min strain data
network = 'IV'
station = 'TSM2'
meta = GtsmMetadata(network,station)
start="2023-11-01T00:00:00"
end = "2023-11-08T00:00:00"
strain_raw = ts_from_mseed(network=network, station=station, location='T0', channel='RS*', start=start, end=end)
strain_raw.stats()

In [ ]:
#convert raw counts to microstrain
name = f"{network}.{station}.gauge.microstrain"
gauge_microstrain = strain_raw.linearize(reference_strains=meta.reference_strains, gap=meta.gap, name=name)
gauge_microstrain.stats()

In [ ]:
#this cell runs hartid to calculate a tidal correction.  If it works, you should see 4 plots of sinusoids
name = f"{network}.{station}.gauge.tide_c"
tide_c = gauge_microstrain.calculate_tide_correction(tidal_parameters=meta.tidal_params, longitude=meta.longitude, name=name)
tide_c.plot()